In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from typing import List, Tuple, Dict
from dataclasses import dataclass
import random
from torch.utils.data import Dataset, DataLoader

In [7]:
#Constants:

FACULTY_NAMES = ['Computer Science', 'Economics', 'Psychology', 'Law', 'Art']

In [5]:
@dataclass
class FacultyParams:
    """Parameters for each faculty"""
    name: str
    utility_vector: np.ndarray  # Hidden vector that determines student success
    capacity: int  # Number of spots available (can be infinite)

@dataclass
class SupplierParams:
    """Parameters for each preparation supplier"""
    name: str
    diff_vector: np.ndarray  # How this supplier modifies student features

class UniversityEnvironment:
    def __init__(
        self,
        n_features: int = 5,  # Number of student features (e.g., math, english, etc.)
        n_faculties: int = 5,  # Number of different faculties
        n_suppliers: int = 4,  # Number of preparation suppliers
        noise_range: Tuple[float, float] = (-5, 5)  # Range for uniform noise
    ):
        self.n_features = n_features
        self.n_faculties = n_faculties
        self.n_suppliers = n_suppliers
        self.noise_range = noise_range
        
        # Initialize faculties with random utility vectors
        self.faculties = [
            FacultyParams(
                name=f"Faculty_{i}",
                utility_vector=np.random.uniform(0, 1, n_features),
                capacity=np.inf  # As per description, infinite capacity
            )
            for i in range(n_faculties)
        ]
        
        # Initialize suppliers with random modification vectors
        self.suppliers = [
          SupplierParams(
              name=f"Supplier_{i}",
              diff_vector=np.array([
                  0.2 if j == idx1 else 0.1 if j == idx2 else 0
                  for j in range(n_features)
              ]),
          )
          for i in range(n_suppliers)
          for idx1, idx2 in [np.random.choice(n_features, size=2, replace=False)]
        ]
        
        self.past_applicants_df = None
        self.current_applicants_df = None

    def generate_past_applicants(
        self,
        n_applicants: int = 1000
    ) -> pd.DataFrame:
        """Generate dataset of past applicants with their outcomes"""
        # Generate random feature vectors
        features = np.random.normal(0, 1, (n_applicants, self.n_features))
        
        # Calculate grades for all faculties
        grades = np.zeros((n_applicants, self.n_faculties))
        # Reshape faculty vectors into (n_faculties, n_features) matrix
        faculty_vectors = np.array([f.utility_vector for f in self.faculties])
        # Calculate base grades using matrix multiplication
        base_grades = np.dot(features, faculty_vectors.T)
        # Add uniform noise to all grades at once
        noise = np.random.uniform(*self.noise_range, size=(n_applicants, self.n_faculties))
        grades = base_grades + noise
        
        # Create DataFrame
        feature_cols = [f"feature_{i}" for i in range(self.n_features)]
        faculty_grade_cols = [f"faculty_{i}_grade" for i in range(self.n_faculties)]
        
        df = pd.DataFrame(features, columns=feature_cols)
        df_grades = pd.DataFrame(grades, columns=faculty_grade_cols)
        
        # Add best faculty (argmax of grades)
        df['assigned_faculty'] = np.argmax(grades, axis=1)
        df['final_grade'] = df_grades.max(axis=1)
        
        self.past_applicants_df = df
        return df

    def generate_current_applicants(
        self,
        n_applicants: int = 100
    ) -> pd.DataFrame:
        """Generate dataset of current applicants"""
        # Generate random feature vectors
        features = np.random.normal(0, 1, (n_applicants, self.n_features))
        
        # Create DataFrame
        feature_cols = [f"feature_{i}" for i in range(self.n_features)]
        df = pd.DataFrame(features, columns=feature_cols)
        
        # Add desired faculty (random)
        df['desired_faculty'] = np.random.randint(0, self.n_faculties, n_applicants)
        
        self.current_applicants_df = df
        return df

    def choose_supplier_for_applicant(
        self,
        applicant_features: np.ndarray,
        desired_faculty: int,
        past_data: pd.DataFrame = None
    ) -> Tuple[int, np.ndarray]:
        """
        Choose the best supplier for an applicant based on past data and supplier effects.
        
        Args:
            applicant_features: The current features of the applicant
            desired_faculty: The faculty index the applicant wants to get into
            past_data: Optional past data to train on. If None, uses self.past_applicants_df
        
        Returns:
            Tuple of (chosen_supplier_idx, modified_features)
        """
        if past_data is None:
            past_data = self.past_applicants_df
        
        if past_data is None:
            raise ValueError("No past data available. Generate past applicants first.")
        
        # Create and train applicant's MLP model
        feature_cols = [f"feature_{i}" for i in range(self.n_features)]
        X_train = torch.FloatTensor(past_data[feature_cols].values)
        y_train = torch.LongTensor(past_data['assigned_faculty'].values)
        
        model = ApplicantMLP(self.n_features, self.n_faculties)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
        
        # Train the model
        model.train()
        for epoch in range(100):  # Quick training, adjust epochs as needed
            optimizer.zero_grad()
            outputs = model(X_train)
            loss = criterion(outputs, y_train)
            loss.backward()
            optimizer.step()
        
        # Evaluate each supplier's effect
        model.eval()
        best_probability = -1
        best_supplier_idx = -1
        best_modified_features = None
        
        original_features = torch.FloatTensor(applicant_features).unsqueeze(0)
        
        with torch.no_grad():
            # Try each supplier
            for i, supplier in enumerate(self.suppliers):
                # Apply supplier's modification
                modified_features = original_features + torch.FloatTensor(supplier.diff_vector)
                
                # Get probability distribution over faculties
                probabilities = model(modified_features)
                
                # Check probability for desired faculty
                prob_desired = probabilities[0, int(desired_faculty)].item()
                
                if prob_desired > best_probability:
                    best_probability = prob_desired
                    best_supplier_idx = i
                    best_modified_features = modified_features.squeeze(0).numpy()
        
        if best_supplier_idx == -1:
            # If no supplier improves probability, return original features with no supplier
            return (-1, applicant_features)
        
        return (best_supplier_idx, best_modified_features)
        
    def recommend(
        self,
        student_features: np.ndarray,
        recommended_faculties: np.ndarray
    ) -> np.ndarray:
        """Calculate final grades for students given their features and recommended faculties
        
        Args:
            student_features: Features matrix of shape (n_students, n_features)
            recommended_faculties: Array of faculty indices of shape (n_students,)
            
        Returns:
            Array of final grades of shape (n_students,)
        """
        # Get utility vectors for all recommended faculties
        faculty_vectors = np.array([self.faculties[f].utility_vector for f in recommended_faculties])
        
        # Calculate base grades using batch matrix multiplication
        base_grades = np.sum(student_features * faculty_vectors, axis=1)
        
        # Generate noise for all students at once
        noise = np.random.uniform(*self.noise_range, size=len(student_features))
        
        return base_grades + noise

    def university_decision_process(
        self,
        current_applicants_features: np.ndarray,
        past_data: pd.DataFrame = None
    ) -> Tuple[np.ndarray, np.ndarray, float]:
        """
        Train university model on past data and make faculty recommendations for current applicants.
        
        Args:
            current_applicants_features: Modified features of current applicants (n_applicants x n_features)
            past_data: Optional past data to train on. If None, uses self.past_applicants_df
        
        Returns:
            Tuple of (chosen_faculties, final_grades, mean_grade)
            - chosen_faculties: Array of faculty indices chosen for each applicant
            - final_grades: Array of final grades received by each applicant
            - mean_grade: Average grade across all applicants
        """
        if past_data is None:
            past_data = self.past_applicants_df
        
        if past_data is None:
            raise ValueError("No past data available. Generate past applicants first.")
        
        # Prepare training data 
        feature_cols = [f"feature_{i}" for i in range(self.n_features)]
        
        X_train = torch.FloatTensor(past_data[feature_cols].values)
        y_train = torch.FloatTensor(past_data['final_grade'].values)
        faculty_train = torch.LongTensor(past_data['assigned_faculty'].values)
        
        # Create and train university model
        model = UniversityMLP(self.n_features, self.n_faculties)
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
        
        # Train the model
        model.train()
        for epoch in range(200):  # More epochs for better training
            optimizer.zero_grad()
            predicted_grades = model(X_train)
            loss = criterion(predicted_grades, y_train)
            loss.backward()
            optimizer.step()
        
        # Make predictions for current applicants
        model.eval()
        with torch.no_grad():
            current_features = torch.FloatTensor(current_applicants_features)
            predicted_grades = model(current_features)
            
            # Choose best faculty for each applicant based on predicted grades
            chosen_faculties = torch.argmax(predicted_grades, dim=1).numpy()
        
        # Get actual final grades using recommend function for all applicants at once
        final_grades = self.recommend(current_applicants_features, chosen_faculties)
        
        mean_grade = np.mean(final_grades)
        
        return chosen_faculties, final_grades, mean_grade

class UniversityMLP(nn.Module):
    """Simple MLP for university decisions"""
    def __init__(self, n_features: int, n_faculties: int):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(n_features, 32),
            nn.ReLU(),
            nn.Linear(32, n_faculties)
        )
    
    def forward(self, x):
        return self.network(x)

class ApplicantMLP(nn.Module):
    """MLP for applicant decisions with softmax output"""
    def __init__(self, n_features: int, n_faculties: int):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(n_features, 32),
            nn.ReLU(),
            nn.Linear(32, n_faculties),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x):
        return self.network(x)

In [3]:
# Example usage:
def run_example():
# Create environment
    env = UniversityEnvironment()
    
    # Generate past applicants
    past_df = env.generate_past_applicants(1000)
    print("Past applicants shape:", past_df.shape)
    
    # Generate current applicants
    current_df = env.generate_current_applicants(100)
    print("Current applicants shape:", current_df.shape)
    
    # Get modified features for all current applicants
    feature_cols = [f"feature_{i}" for i in range(env.n_features)]
    modified_features = []
    
    for idx in range(len(current_df)):
        student_features = current_df.iloc[idx][feature_cols].values
        desired_faculty = current_df.iloc[idx]['desired_faculty']
        
        _, modified_student_features = env.choose_supplier_for_applicant(
            student_features,
            desired_faculty
        )
        modified_features.append(modified_student_features)
    
    modified_features = np.array(modified_features)
    
    # Run university decision process
    chosen_faculties, final_grades, mean_grade = env.university_decision_process(modified_features)
    
    # Print results
    print("\nResults:")
    print(f"Mean grade across all applicants: {mean_grade:.2f}")
    
    # Print detailed results for first 5 applicants
    print("\nDetailed results for first 5 applicants:")
    for i in range(5):
        desired_faculty = current_df.iloc[i]['desired_faculty']
        print(f"\nApplicant {i}:")
        print(f"Desired faculty: {desired_faculty}")
        print(f"Assigned faculty: {chosen_faculties[i]}")
        print(f"Final grade: {final_grades[i]:.2f}")

In [6]:
run_example()

Past applicants shape: (1000, 7)
Current applicants shape: (100, 6)


KeyError: "None of [Index(['faculty_0_grade', 'faculty_1_grade', 'faculty_2_grade'], dtype='object')] are in the [columns]"